In [0]:
%%sh
raw_data_path = s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2018-01-04/platform=1/
unified_data_path_legacy = s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/
raw_domain_path = s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_BASIC_METRICS/version=v1.0.0/
aws s3 ls  s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

select *  from store_est_category_fact_v1 where country_code='US'and device_code in ('ios-phone') 
and date between '2020-09-01' and '2020-09-30' and app_id = 1434655572 and category_id = 100003 and country_code = 'US';
EOF

In [0]:

unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/granularity_code=weekly/date=2020-09-05/"
category_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=app/granularity_code=weekly/date=2020-09-05/"
basic_raw_path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=WEEK/date=2020-09-05/platform=1/"
spark.read.format("delta").load(unified_path).createOrReplaceTempView("test_unified")
spark.read.format("delta").load(category_path).createOrReplaceTempView("test_category")
# spark.read.format("parquet").load(basic_raw_path).createOrReplaceTempView("test_raw")
# spark.sql("select count(distinct product_key) as basic_count from test_unified ").show()
# spark.sql("select count(1) as category_count from test_category where country_code <> 'ROW'").show()
# spark.sql("select * from test_raw where app_id=20600013700573 ").show(10, False)
# spark.sql("select product_key, count(1) as count from test_category where category_key = 400013 and device_code = 'android-phone'  group by product_key order by product_key desc ").show(1000, False)
spark.sql("select distinct country_code from test_category where device_code = 'android-phone' and product_key = 20600013700573 and category_key = 400013").show(1000, False)
# spark.sql("select sum(est_active_users), sum(est_population) from test_unified where product_type_code='app' and granularity_code='monthly' and country_code <> 'ROW'").show()
# spark.sql("select count(distinct product_key, device_code, country_code, granularity_code, date) as basic_distinct_count from test_unified").show()

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2020, 8, 1)
end_date = datetime(2020, 9, 30)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()

print DATE_GRANULARITY_MAPPINGLIST["monthly"]
print DATE_GRANULARITY_MAPPINGLIST["weekly"]
print DATE_GRANULARITY_MAPPINGLIST["daily"]

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from pyspark.sql import Row
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, DoubleType, ShortType

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)
date_count_sql =  """select  sum(count) from plproxy.execute_select($proxy$
SELECT count(1) as count
FROM mu.category_{granularity}_{device} where 
date = '{date}' and category_id = {legacy_category_id} $proxy$)
 t (count bigint) ;
"""

domain_count_sql = """
select  sum(count) from plproxy.execute_select($proxy$
SELECT count(1) as count
FROM mw.category_{simple_granularity}_{device_code} where 
date = '{date}' and category_id = {unified_category_id} $proxy$)
 t (count bigint) ;
"""


# granularity_list = ['daily', 'weekly', 'monthly']
granularity_list = ['daily']
device_list = [1001, 1002, 2001, 2002]
ios_device_list = [ 2001, 2002]
gp_device_list = [1001, 1002]
device_code_list = ['android-phone', 'android-tablet', 'ios-phone', 'ios-tablet']
# device_code_list = ['ios-tablet']
device_mapping = {1001:'android-phone', 1002:'android-tablet', 2001:'ios-phone', 2002:'ios-tablet'}


legacy_ios_category_list = [6016, 6017, 6018, 6020, 6021, 6022, 6023, 6024, 36, 6001, 7001, 7002, 7003, 7004, 7005, 7006, 7007, 7008, 7009, 7010, 7011, 7012, 7013, 7014, 7015, 7016, 7017, 7018, 7019, 100, 360, 361, 362, 6000, 363, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015]

# legacy_ios_category_list = [6016, 6017, 6014]

ios_category_mapping = {6016: 800022, 6017: 800021, 6018: 800019, 6020: 800028, 6021: 800030, 6022: 800019, 6023: 800024, 6024: 800033, 36: 800000, 6001: 800038, 7001: 800002, 7002: 800003, 7003: 800002, 7004: 800004, 7005: 800006, 7006: 800006, 7007: 800006, 7008: 800007, 7009: 800008, 7010: 800008, 7011: 800010, 7012: 800005, 7013: 800011, 7014: 800012, 7015: 800013, 7016: 800014, 7017: 800012, 7018: 800015, 7019: 800016, 100: 800018, 360: 800041, 361: 800042, 362: 800043, 6000: 800020, 363: 800044, 6002: 800036, 6003: 800037, 6004: 800035, 6005: 800034, 6006: 800019, 6007: 800032, 6008: 800031, 6009: 800030, 6010: 800037, 6011: 800029, 6012: 800026, 6013: 800025, 6014: 800001, 6015: 800023}

legacy_gp_category_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75]

gp_category_mapping = {1: 800000, 2: 800001, 3: 800002, 4: 800005, 5: 800006, 6: 800005, 7: 800009, 8: 800011, 9: 800014, 10: 800017, 11: 800018, 12: 800019, 13: 800020, 14: 800019, 15: 800034, 16: 800021, 17: 800022, 18: 800023, 19: 800025, 20: 800019, 21: 800026, 22: 800027, 23: 800031, 24: 800028, 25: 800029, 26: 800030, 27: 800036, 28: 800031, 29: 800032, 30: 800033, 31: 800034, 32: 800035, 33: 800036, 34: 800037, 35: 800037, 36: 800038, 37: 800040, 38: 800002, 39: 800003, 40: 800016, 41: 800002, 42: 800004, 43: 800006, 44: 800006, 46: 800007, 47: 800008, 48: 800010, 49: 800005, 51: 800012, 52: 800013, 54: 800012, 55: 800015, 56: 800041, 57: 800042, 58: 800043, 59: 800044, 60: 800041, 61: 800041, 62: 800041, 63: 800041, 64: 800041, 65: 800041, 66: 800039, 67: 800032, 68: 800026, 69: 800033, 70: 800026, 71: 800026, 72: 800022, 73: 800024, 75: 800025}

legacy_to_individual_ios = {36: 100000, 6014: 100001, 7001: 100002, 7002: 100003, 7003: 100004, 7004: 100005, 7005: 100006, 7006: 100007, 7007: 100008, 7008: 100009, 7009: 100010, 7010: 100011, 7011: 100012, 7012: 100013, 7013: 100014, 7014: 100015, 7015: 100016, 7016: 100017, 7017: 100018, 7018: 100019, 7019: 100020, 100: 100021, 6018: 100022, 6000: 100023, 6022: 100024, 6017: 100025, 6016: 100026, 6015: 100027, 6023: 100028, 6013: 100029, 360: 100030, 361: 100031, 362: 100032, 363: 100033, 6012: 100034, 6021: 100035, 6020: 100064, 6011: 100065, 6010: 100066, 6009: 100067, 6008: 100068, 6007: 100069, 6006: 100070, 6024: 100071, 6005: 100072, 6004: 100073, 6003: 100075, 6002: 100076, 6001: 100077}

unified_category_list = [800000, 800001, 800002, 800003, 800004, 800005, 800006, 800007, 800008, 800009, 800010, 800011, 800012, 800013, 800014, 800015, 800016, 800017, 800018, 800019, 800020, 800021, 800022, 800023, 800024, 800025, 800026, 800027, 800028, 800029, 800030, 800031, 800032, 800033, 800034, 800035, 800036, 800037, 800038, 800039, 800040, 800041, 800042, 800043, 800044]


def get_plproxy_count(sql_str=date_count_sql):
    for device in ios_device_list:
        for granularity in granularity_list:
            for date in DATE_GRANULARITY_MAPPINGLIST[granularity]:
                for category in legacy_ios_category_list:
                    result = query(PLPROXY_DSN, sql_str.format(granularity=granularity, date=date, device=device, legacy_category_id=category))
                    print device_mapping[device], granularity, date, category, result[0][0]


def get_unified_result():
    category_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=app/granularity_code={granularity}/date={date}/"
    for device in ios_device_list:
        for granularity in granularity_list:
            for date in DATE_GRANULARITY_MAPPINGLIST[granularity]:
                for category in legacy_ios_category_list:
                    spark.read.format("delta").load(category_path.format(granularity=granularity, date=date)).createOrReplaceTempView("test_category")
                    df = spark.sql("select count(1) as count from test_category where device_code = '{device}' and category_key = {category} and country_code <> 'ROW'".format(device=device_mapping[device], category=legacy_to_individual_ios[category])).collect()
                    # print df[0][0]
                    print device_mapping[device], granularity, date, category, df[0][0]
                    
simple_device_list = ['ip', 'ap']
# simple_granularity_list = ['m', 'w', 'd' ]
simple_granularity_list = ['m' ]
granularity_mapping = {'d':'daily', 'w':'weekly', 'm': 'monthly'}

def get_domain_plproxy_count(sql_str=domain_count_sql):
    for simple_device in simple_device_list:
        for simple_granularity in simple_granularity_list:
            for date in DATE_GRANULARITY_MAPPINGLIST[granularity_mapping[simple_granularity]]:
                for unified_category in unified_category_list:
                    result = query(PLPROXY_DSN, sql_str.format(simple_granularity=simple_granularity, device_code=simple_device, date=date,  unified_category_id=unified_category))
                    print simple_device, simple_granularity, date, unified_category, result[0][0]
        
def get_domain_unified_result():
    phone_list = ['ios-phone', 'android-phone']
    category_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=domain/granularity_code={granularity}/date={date}/"
    for device in phone_list:
        for simple_granularity in simple_granularity_list:
            for date in DATE_GRANULARITY_MAPPINGLIST[granularity_mapping[simple_granularity]]:
                for unified_category in unified_category_list:
                    spark.read.format("delta").load(category_path.format(granularity=granularity_mapping[simple_granularity], date=date)).createOrReplaceTempView("test_category")
                    df = spark.sql("select count(1) as count from test_category where device_code = '{device}' and unified_category_key = {unified_category} and country_code <> 'ROW'".format(device=device, unified_category=unified_category)).collect()
                    print device, simple_granularity, date, unified_category, df[0][0]


                
def get_unified_tmp(date, device_code, individual_category_id, granularity):
    category_path_tmp = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=app/granularity_code={granularity}/date={date}/".format(granularity=granularity, date=date)
    spark.read.format("delta").load(category_path_tmp).createOrReplaceTempView("test_category_tmp")
    spark.sql("select distinct product_key as app_id from test_category_tmp where device_code = '{device_code}' and category_key = {individual_category_id} and country_code <> 'ROW'".format(device_code=device_code, individual_category_id=individual_category_id)).createOrReplaceTempView("unified_app")
    # spark.sql("select distinct country_code from test_category_tmp where device_code = 'ios-tablet'   and product_key = 284882215").show(100, False)
            
            
def get_plproxy_tmp(date, device_id, legacy_category_id, granularity):
    date_count_sql_tmp =  """select  distinct app_id from plproxy.execute_select($proxy$
SELECT distinct app_id as app_id
FROM mu.category_{granularity}_{device_id} where 
date = '{date}' and category_id in ({legacy_category_id}) $proxy$)
 t (app_id bigint);
""".format(device_id=device_id, date=date, legacy_category_id=legacy_category_id, granularity=granularity)
    print date_count_sql_tmp
    result = query(PLPROXY_DSN, date_count_sql_tmp)
    df_data = [Row(app_id=r[0]) for r in result]
    _schema =StructType([StructField("app_id", LongType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy
    # print result[0][0]
    
def get_main_category(app_view):
    path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/"
    spark.read.format("parquet").load(path).createOrReplaceTempView("test")
    spark.sql("select id, last_updated, release_date,current_release_date, category_id  from test where id in  (select app_id from {app_view})".format(app_view=app_view)).show(100, False)
    
def get_rank_category(app_view):
    rank_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=monthly/date=2020-09-30/"
    spark.read.format("parquet").load(rank_path).createOrReplaceTempView("rank_category")
    spark.sql("select * from rank_category where app_id = 1477487725 limit 10").show(10, False)
    spark.sql("select distinct app_id, category_id from rank_category where app_id in (select app_id from {app_view}) and category_id = 100004".format(app_view=app_view)).show(100, False)
    
def compare():
    date = '2020-08-22'
    device_id = 1001
    device_code = 'android-phone'
    legacy_category_id = 57
    individual_category_id = 400063
    granularity = 'weekly'
    df_plproxy = get_plproxy_tmp(date, device_id, legacy_category_id, granularity)
    get_unified_tmp(date, device_code, individual_category_id, granularity)
    df_plproxy.createOrReplaceTempView("plproxy_app")
    spark.sql("select app_id from plproxy_app except select app_id from unified_app").createOrReplaceTempView("plproxy_diff_unified")
    spark.sql("select app_id from unified_app except select app_id from plproxy_app ").createOrReplaceTempView("unifid_diff_plproxy")
    
    spark.sql("select app_id as plproxy_diff_unified from plproxy_diff_unified").show(100, False)
    spark.sql("select app_id as unifid_diff_plproxy from unifid_diff_plproxy").show(100, False)
    
    get_main_category("plproxy_diff_unified")
    get_main_category("unifid_diff_plproxy")
    
    
    
# get_rank_category("unifid_diff_plproxy")
get_plproxy_count()

    

In [0]:


path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/"
spark.read.format("parquet").load(path).createOrReplaceTempView("test")
spark.sql("select id, last_updated, release_date,current_release_date, category_id  from test where id in  (20600011762403)").show(100, False)

In [0]:

spark.sql("select * from test_category_tmp where device_code = 'ios-tablet' and category_key=100005 and product_key in (1137491018, 341532096,1126032958)").show(100, False)

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from pyspark.sql import Row
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, DoubleType, ShortType

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

legacy_ios_category_list = [6016, 6017, 6018, 6020, 6021, 6022, 6023, 6024, 36, 6001, 7001, 7002, 7003, 7004, 7005, 7006, 7007, 7008, 7009, 7010, 7011, 7012, 7013, 7014, 7015, 7016, 7017, 7018, 7019, 100, 360, 361, 362, 6000, 363, 6002, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010, 6011, 6012, 6013, 6014, 6015]
ios_category_mapping = {6016: 800022, 6017: 800021, 6018: 800019, 6020: 800028, 6021: 800030, 6022: 800019, 6023: 800024, 6024: 800033, 36: 800000, 6001: 800038, 7001: 800002, 7002: 800003, 7003: 800002, 7004: 800004, 7005: 800006, 7006: 800006, 7007: 800006, 7008: 800007, 7009: 800008, 7010: 800008, 7011: 800010, 7012: 800005, 7013: 800011, 7014: 800012, 7015: 800013, 7016: 800014, 7017: 800012, 7018: 800015, 7019: 800016, 100: 800018, 360: 800041, 361: 800042, 362: 800043, 6000: 800020, 363: 800044, 6002: 800036, 6003: 800037, 6004: 800035, 6005: 800034, 6006: 800019, 6007: 800032, 6008: 800031, 6009: 800030, 6010: 800037, 6011: 800029, 6012: 800026, 6013: 800025, 6014: 800001, 6015: 800023}

legacy_gp_category_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75]

gp_category_mapping = {1: 800000, 2: 800001, 3: 800002, 4: 800005, 5: 800006, 6: 800005, 7: 800009, 8: 800011, 9: 800014, 10: 800017, 11: 800018, 12: 800019, 13: 800020, 14: 800019, 15: 800034, 16: 800021, 17: 800022, 18: 800023, 19: 800025, 20: 800019, 21: 800026, 22: 800027, 23: 800031, 24: 800028, 25: 800029, 26: 800030, 27: 800036, 28: 800031, 29: 800032, 30: 800033, 31: 800034, 32: 800035, 33: 800036, 34: 800037, 35: 800037, 36: 800038, 37: 800040, 38: 800002, 39: 800003, 40: 800016, 41: 800002, 42: 800004, 43: 800006, 44: 800006, 46: 800007, 47: 800008, 48: 800010, 49: 800005, 51: 800012, 52: 800013, 54: 800012, 55: 800015, 56: 800041, 57: 800042, 58: 800043, 59: 800044, 60: 800041, 61: 800041, 62: 800041, 63: 800041, 64: 800041, 65: 800041, 66: 800039, 67: 800032, 68: 800026, 69: 800033, 70: 800026, 71: 800026, 72: 800022, 73: 800024, 75: 800025}


legacy_category_list = [36, 6014, 7012, 6004, 6008]
# legacy_gp_category_list = [1, 2, 44, 13, 63]
legacy_gp_category_list = [13]
gp_category_mapping = {1: 400000,
    2: 400001,
    44: 400007,
    13: 400031,
    63: 400068
    
}
# legacy_category_list = [6014]
device_id_list = [1001, 1002, 2001, 2002]
category_mapping = {
    36: 800000,
    6014:800001,
    7012:800005,
    6004:800035,
    6008:800031
}
unified_category_list = [800000, 800001, 800005, 800035, 800031]
# unified_category_list = [800001]

aggr_sql = """select  app_id, rank, kpi, estimate from plproxy.execute_select($proxy$
SELECT app_id, rank, kpi, estimate
FROM mu.category_weekly_2001_143441 where 
date in ('2020-09-05', '2020-09-12') 
and rank <= 1000 
and category_id = {legacy_category_id} $proxy$)
 t (app_id bigint, rank integer, kpi smallint, estimate double precision) order by rank asc;
"""


def get_plproxy_result(sql_str):
    plproxy_result = []
    result = query(PLPROXY_DSN, sql_str)
    df_data = [Row(app_id=r[0], rank=r[1], kpi=r[2], estimate=r[3]) for r in result]
    _schema =StructType([StructField("app_id", LongType(), False), 
    StructField("rank", IntegerType(), False),
    StructField("kpi", ShortType(), False),
    StructField("estimate", DoubleType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy
    
def get_plproxy_domain_result(sql_str):
    plproxy_result = []
    result = query(PLPROXY_DSN, sql_str)
    df_data = [Row(domain_id=r[0], est_usage_penetration=r[1], rank_est_usage_penetration=r[2]) for r in result]
    _schema =StructType([StructField("domain_id", LongType(), False), 
    StructField("est_usage_penetration", DoubleType(), False),
    StructField("rank_est_usage_penetration", IntegerType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy

def get_unified_data():
    domain_unified_source_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=w/month=202005/date=2020-05-16"
    
    spark.read.format("delta").load(unified_source_path).createOrReplaceTempView("test_unified")
    spark.sql("select distinct domain_id from test_unified where  est_average_active_users <> 0 and est_average_active_users is not null order by domain_id asc").createOrReplaceTempView("unified_df_new")

def get_plproxy_data():
     df_plproxy=get_plproxy_result()
     df_plproxy.createOrReplaceTempView("plproxy_df_new")
     spark.sql("select count(distinct app_id) from plproxy_df_new").show(10, False)
    #  spark.sql("select * from plproxy_df_new order by rank asc").show(10000, False)
     
     spark.sql("select app_id, avg(estimate) as estimate from plproxy_df_new group by app_id order by estimate desc").show(10000, False)



def compare_single():
    single_sql =  """select  app_id, rank, kpi, estimate from plproxy.execute_select($proxy$
SELECT app_id, rank, kpi, estimate
FROM mu.category_weekly_1001_1000 where 
date = '2020-09-12' 
and rank <= 1000 
and category_id = {legacy_category_id} $proxy$)
 t (app_id bigint, rank integer, kpi smallint, estimate double precision) order by rank asc;
"""
    for category in legacy_gp_category_list:
        df_plproxy=get_plproxy_result(single_sql.format(legacy_category_id=category))
        df_plproxy.createOrReplaceTempView("plproxy_df")
        spark.sql("select product_key as app_id from basic_with_category_weekly where date = '2020-09-12' and device_code = 'android-phone' and country_code='WW' and category_key={category} order by est_usage_penetration desc limit 1000".format(category=gp_category_mapping[category])).createOrReplaceTempView("apps_new")
        spark.sql("select app_id from plproxy_df except select app_id from apps_new").createOrReplaceTempView("plproxy_diff_new")
        spark.sql("select app_id from apps_new except select app_id from plproxy_df").createOrReplaceTempView("new_diff_plproxy")
        # print "category is {category}".format(category=category)
        spark.sql("select app_id as plproxy_diff_new from plproxy_diff_new").show(1000, False)
        spark.sql("select app_id as new_diff_plproxy from new_diff_plproxy").show(1000, False)
        
        spark.sql("select count(1) as plproxy from plproxy_df").show(1000, False)
        spark.sql("select count(1) as new from apps_new").show(1000, False)
        
        # plproxy_diff_new = spark.sql("select count(1) as plproxy_diff_snowflake from plproxy_diff_new").collect()
        # new_diff_plproxy = spark.sql("select count(1) as snowflake_diff_plproxy from new_diff_plproxy").collect()
        # print "category", category, "plproxy_diff_new", plproxy_diff_new[0][0], "new_diff_plproxy", new_diff_plproxy[0][0]

def compare_aggr():
    # basic_with_category_{granularity}; .format(date='2020-08-31', legacy_category_id=36)
    
    for category in legacy_category_list:
        df_plproxy=get_plproxy_result(aggr_sql.format(legacy_category_id=category))
        df_plproxy.createOrReplaceTempView("plproxy_df")
        spark.sql("select app_id, max(estimate) as estimate from plproxy_df group by app_id order by estimate desc limit 1000").createOrReplaceTempView("plproxy_aggr")
        
        spark.sql("select product_key as app_id, sum(est_active_users)/sum(est_population) as aggr_up from basic_with_category_weekly where date in  ('2020-08-29', '2020-08-22') and device_code = 'ios-phone' and country_code='US' and unified_category_key={unified_category} group by app_id order by aggr_up desc limit 1000".format(unified_category=category_mapping[category])).createOrReplaceTempView("apps_new")
        spark.sql("select app_id from plproxy_aggr except select app_id from apps_new").createOrReplaceTempView("plproxy_diff_new")
        spark.sql("select app_id from apps_new except select app_id from plproxy_aggr").createOrReplaceTempView("new_diff_plproxy")
        # print "category is {category}".format(category=category)
        spark.sql("select app_id as plproxy_diff_new from plproxy_diff_new order by app_id desc").show(1000, False)
        spark.sql("select app_id as new_diff_plproxy from new_diff_plproxy order by app_id desc").show(1000, False)
        
        spark.sql("select app_id as plproxy_diff_new from plproxy_diff_new order by app_id desc").show(1000, False)
        spark.sql("select app_id as new_diff_plproxy from new_diff_plproxy order by app_id desc").show(1000, False)
        
        # plproxy_diff_new = spark.sql("select count(1) as plproxy_diff_new from plproxy_diff_new").collect()
        # new_diff_plproxy = spark.sql("select count(1) as new_diff_plproxy from new_diff_plproxy").collect()
        # print "category", category, "plproxy_diff_new", plproxy_diff_new[0][0], "new_diff_plproxy", new_diff_plproxy[0][0]
        
def compare_single_domain():
    domain_single_sql = """
select  domain_id, est_usage_penetration, rank_est_usage_penetration from plproxy.execute_select($proxy$
SELECT domain_id, est_usage_penetration, rank_est_usage_penetration
FROM mw.category_m_ap where 
date = '2020-08-31' and category_id = {unified_category_id} and country_code = 'us' and rank_est_usage_penetration < 1000 $proxy$)
 t (domain_id bigint, est_usage_penetration real, rank_est_usage_penetration int) ;
""" 
    for category in unified_category_list:
        df_plproxy=get_plproxy_domain_result(domain_single_sql.format(unified_category_id=category))
        df_plproxy.createOrReplaceTempView("plproxy_df")
        spark.sql("select product_key as domain_id from basic_with_category_monthly where date = '2020-08-31' and device_code = 'android-phone' and country_code='US' and unified_category_key={category} order by est_usage_penetration desc limit 1000".format(category=category)).createOrReplaceTempView("domain_new")
        spark.sql("select domain_id from plproxy_df except select domain_id from domain_new").createOrReplaceTempView("plproxy_diff_new")
        spark.sql("select domain_id from domain_new except select domain_id from plproxy_df").createOrReplaceTempView("new_diff_plproxy")
        # print "category is {category}".format(category=category)
        # spark.sql("select count(distinct domain_id) from domain_new").show(1000, False)
        # spark.sql("select count(1) as new_diff_plproxy from new_diff_plproxy").show(1000, False)
        
        
        plproxy_diff_new = spark.sql("select count(1) as plproxy_diff_new from plproxy_diff_new").collect()
        new_diff_plproxy = spark.sql("select count(1) as new_diff_plproxy from new_diff_plproxy").collect()
        print "category", category, "plproxy_diff_new", plproxy_diff_new[0][0], "new_diff_plproxy", new_diff_plproxy[0][0]
        
def compare_domain_aggr():
    domain_aggr_sql = """
select  domain_id, est_usage_penetration, rank_est_usage_penetration from plproxy.execute_select($proxy$
SELECT domain_id, est_usage_penetration, rank_est_usage_penetration
FROM mw.category_w_ip_us where 
date in ('2020-08-15', '2020-08-22') and category_id = {unified_category_id} and rank_est_usage_penetration < 1000 $proxy$)
 t (domain_id bigint, est_usage_penetration real, rank_est_usage_penetration int) ;
""" 
    for category in unified_category_list:
        df_plproxy=get_plproxy_domain_result(domain_aggr_sql.format(unified_category_id=category))
        df_plproxy.createOrReplaceTempView("plproxy_df")
        spark.sql("select domain_id, max(est_usage_penetration) as est_usage_penetration from plproxy_df group by domain_id order by est_usage_penetration desc limit 1000").createOrReplaceTempView("plproxy_aggr")
        
        spark.sql("select product_key as domain_id, sum(est_active_users)/sum(est_population) as aggr_up from basic_with_category_weekly where date in  ('2020-08-15', '2020-08-22') and device_code = 'ios-phone' and country_code='US' and unified_category_key={unified_category} group by product_key order by aggr_up desc limit 1000".format(unified_category=category)).createOrReplaceTempView("domain_new")
        # spark.sql("select count(1) from domain_new").show(10, False)
        # spark.sql("select count(1) from plproxy_aggr").show(10, False)
        spark.sql("select domain_id from plproxy_aggr except select domain_id from domain_new").createOrReplaceTempView("plproxy_diff_new")
        spark.sql("select domain_id from domain_new except select domain_id from plproxy_aggr").createOrReplaceTempView("new_diff_plproxy")
        
        plproxy_diff_new = spark.sql("select count(1) as plproxy_diff_new from plproxy_diff_new").collect()
        new_diff_plproxy = spark.sql("select count(1) as new_diff_plproxy from new_diff_plproxy").collect()
        print "category", category, "plproxy_diff_new", plproxy_diff_new[0][0], "new_diff_plproxy", new_diff_plproxy[0][0]
    
    
compare_domain_aggr()
# unified_source_path = "s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.basic.v4/fact/granularity=w/date=2020-05-16"
# spark.read.format("delta").load(unified_source_path).show(10)


In [0]:

# spark.sql("select * from basic_with_category_weekly where date in  ('2020-08-29', '2020-08-22') and device_code = 'ios-phone' and country_code='US' and unified_category_key=800001").createOrReplaceTempView("new_data")

spark.sql("select  product_key,est_usage_penetration from new_data where product_key in (select app_id from new_diff_plproxy) and date = '2020-08-22' order by product_key desc").show(100, False)

       

In [0]:

raw_data_path =  "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2020-08-01/"
usage_basic_unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.app-basic-kpi.v8/fact/product_type_name=app/granularity=monthly/"
raw_domain_path = ""
spark.read.option("basePath", "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0").parquet(raw_data_path).createOrReplaceTempView("test")
spark.read.format("delta").load(usage_basic_unified_path).createOrReplaceTempView("test_unified")
spark.sql("select count(distinct country_code,device_code,date,product_id,market_code,granularity) from test_unified where est_active_users is not null and est_active_users <> 0").createOrReplaceTempView("unified_remove_category")
# spark.read.format("delta").load(raw_data_path).createOrReplaceTempView("test")

# distint app count
# spark.sql("select product_id from test_unified where unified_category_key = 0 limit 10").show()
spark.sql("select distinct app_id from test").createOrReplaceTempView("raw_app_list")
spark.sql("select distinct product_id from test_unified").createOrReplaceTempView("unified_app_list")
# spark.sql("select * from unified_app_list limit 10").show()
spark.sql("select app_id from raw_app_list EXCEPT select product_id from unified_app_list").createOrReplaceTempView("app_diff")
spark.sql("select count(*) from app_diff").show()
# spark.sql("select * from app_diff order by app_id asc").show()
# spark.sql("select * from test where app_id = 20600000328027").show()
# spark.sql("select * from test_unified where product_id = 20600000328027").show()
# device_count
spark.sql("select count(*)  as raw_line_count from test where AU is not null and AU <> 0").show()
spark.sql("select count(*)  as unified_line_count from unified_remove_category").show()


In [0]:

granularity_list = ["monthly", "weekly", "daily"]
category_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.category.v6/dimension/product_type_code=domain/"
usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=domain/"
for granularity in granularity_list:
    filter_str = "date between '2020-08-01' and '2020-08-31' and granularity_code = '{granularity}'".format(granularity=granularity)
    print filter_str
    spark.read.format("delta").load(category_path).filter(filter_str).createOrReplaceTempView("category_view")
    spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str).createOrReplaceTempView("basic_view")
    spark.sql("""
select basic.DATE_KEY,
basic.DEVICE_CODE,
basic.COUNTRY_CODE,basic.COUNTRY_KEY,
basic.date,
basic.DEVICE_KEY,basic.GRANULARITY_CODE,basic.GRANULARITY_KEY,basic.MARKET_CODE,
basic.MARKET_KEY,basic.PRODUCT_KEY,basic.PRODUCT_TYPE_CODE,basic.PRODUCT_TYPE_KEY,
basic.EST_TOTAL_TIME_MILLISECONDS_OF_MAIN_CATEGORY,basic.EST_WIFI_BYTES,basic.EST_ACTIVE_USERS,
basic.EST_TOTAL_BYTES,basic.EST_TOTAL_SESSION_COUNT_OF_MAIN_CATEGORY,
basic.EST_TOTAL_BYTES_OF_MAIN_CATEGORY,basic.EST_TOTAL_ACTIVE_DAYS,basic.EST_INSTALL_BASE,
basic.EST_POPULATION,basic.EST_TOTAL_TIME_MILLISECONDS,basic.EST_TOTAL_SESSION_COUNT,
basic.DEVICE_FORM_FACTOR_CODE,basic.DEVICE_FORM_FACTOR_KEY,basic.PARENT_DEVICE_CODE,basic.PARENT_DEVICE_KEY,basic.PLATFORM_CODE,basic.PLATFORM_KEY,
basic.est_usage_penetration,
map.category_key as category_key,
map.unified_category_key as unified_category_key
from basic_view basic
join category_view map
on basic.date = map.date
and basic.product_key = map.product_key
and basic.granularity_code = map.granularity_code
and basic.country_code = map.country_code
and basic.device_code = map.device_code
    """).createOrReplaceTempView("basic_with_category_{granularity}".format(granularity=granularity))
    # spark.sql("select count(1) from category_view").show(1, False)
    # spark.sql("select count(1) from basic_with_category_{granularity}".format(granularity=granularity)).show(1, False)
    


In [0]:

granularity_list = ["monthly", "weekly", "daily"]

usage_basic_after_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/"
usage_basic_before_transform = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/"

for granularity in granularity_list:
    filter_str = "date between '2020-08-01' and '2020-08-31' and granularity = '{granularity}' and est_average_active_users is not null and est_average_active_users <> 0".format(granularity=granularity)
    filter_str_after = "date between '2020-08-01' and '2020-08-31' and granularity_code = '{granularity}' and est_active_users is not null and est_active_users <> 0".format(granularity=granularity)
    spark.read.format("parquet").load(usage_basic_before_transform).filter(filter_str).createOrReplaceTempView("before_trans")
    spark.read.format("delta").load(usage_basic_after_transform).filter(filter_str_after).createOrReplaceTempView("after_trans")
    spark.sql("select date, count(1) as before_trans from before_trans group by date order by date desc").show(100, False)
    spark.sql("select date, count(1) as after_trans from after_trans where country_code not in ('ROW') group by date order by date desc").show(100, False)

# df1 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact").parquet(usage_basic_unified_path).createOrReplaceTempView("test")
# spark.read.format("delta").load(usage_basic_unified_path).createOrReplaceTempView("test")
# spark.sql("select * from test limit 1").printSchema()

# spark.sql("select distinct(country_code,device_code,date,product_id,market_code,granularity) from test group by country_code,device_code,date,product_id,market_code,granularity").createOrReplaceTempView("unified_remove_category")
# spark.sql("select count(*) from unified_remove_category").show()


In [0]:

spark.read.format("delta").load(usage_basic_after_transform).filter("date between '2020-08-01' and '2020-08-31' and granularity_code = 'monthly' and est_active_users is not null and est_active_users <> 0").createOrReplaceTempView("after_trans")
spark.sql("select sum(est_active_users), sum(est_usage_penetration), sum(est_open_rate), sum(est_total_session_count)/sum(est_active_users) as est_average_session_count_per_user  from after_trans where country_code not in ('WW', 'CN') and product_key = '284882215' and device_code = 'ios-phone' group by product_key").show(10, False)
spark.sql("select est_active_users, est_usage_penetration, est_open_rate, est_average_session_count_per_user from after_trans where country_code in ('WW') and product_key = '284882215' and device_code = 'ios-phone' ").show(10, False)

In [0]:

spark.sql("select count(1) from after_trans where est_active_users is null or est_active_users == 0").show()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v6/fact/product_type_code=app/

In [0]:
%%sh
# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
# internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
            select  app_id, rank, kpi, estimate from plproxy.execute_select(\$proxy\$
SELECT app_id, rank, kpi, estimate FROM mu.category_daily where date = '2020-08-01' and rank <= 1000 and category_id = 36
and store_id = 143441 and device_id = 2001 \$proxy\$)
 t (app_id bigint, rank integer, kpi smallint, estimate double precision) order by rank asc;


EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF
            select  distinct category_id from plproxy.execute_select(\$proxy\$
SELECT distinct category_id
FROM aa.app_store_daily_estimate_1_143441_202008 where
date between '2020-08-01' and '2020-08-31'
and app_id in (1487212912)
and category_id = 7012 and feed_id in (0,1,2)\$proxy\$)
 t (category_id int );

EOF